In [10]:
import os

cwd = os.getcwd()

if cwd.endswith('prototype'):
    PATH = os.path.join('..', 'data', 'handwritten-hanacaraka')
    os.chdir(PATH)
    new_cwd = os.getcwd()

    print(f"The current working directory was: {cwd}")
    print(f"It has been changed to: {new_cwd}")

else:
    print(f"The current working directory is: {cwd}")

The current working directory is: /home/helmyl/hanacaraka-classifier/data/handwritten-hanacaraka


In [11]:
from PIL import Image

subdirs = [d for d in os.listdir() if os.path.isdir(d)]

image_files = []
for subdir in subdirs:
    image_files.extend([os.path.join(subdir, f) for f in os.listdir(subdir) if f.endswith('.png')])

print(f'Number of image files: {len(image_files)}\n')

corrupted_files = []
for image_file in image_files:
    try:
        img = Image.open(image_file)
    except:
        corrupted_files.append(image_file)
        
print(f'Number of corrupted files: {len(corrupted_files)}\n')
print(f'Corrupted files: {os.linesep.join(corrupted_files)}')

Number of image files: 1579

Number of corrupted files: 17

Corrupted files: dha/4696038101413220221_base64_32.png
dha/4697111846216686148_base64_32.png
ga/4696394596413810473_base64_43.png
ha/4694803355817399930_base64_5.png
ja/4694023059412567494_base64_34.png
la/4695952290615665559_base64_27.png
nya/4695963539838682931_base64_38.png
pa/4694579845264951891_base64_30.png
pa/4696000994938205989_base64_30.png
ra/4694221966812671068_base64_14.png
ra/4695660931512476490_base64_14.png
ra/4695874552811358083_base64_14.png
ta/4693403521314974166_base64_21.png
ta/4694044299313201617_base64_21.png
ta/4694584611982169565_base64_21.png
tha/4695963539838682931_base64_47.png
tha/4696273780025212257_base64_47.png


In [15]:
import shutil
import os

new_dir = os.path.join('..', 'renamed_images')

for image_file in image_files:
    if image_file in corrupted_files:
        continue
    
    subdir, filename = image_file.split('/')
    subdir_path = os.path.join(new_dir, subdir)
    os.makedirs(subdir_path, exist_ok=True)
    new_filename = f'{subdir}_{len(os.listdir(subdir_path)) + 1}.png'
    shutil.copy(image_file, os.path.join(subdir_path, new_filename))

print(f'number of image files: {len(image_files) - len(corrupted_files)}')

number of image files: 1562
